In [1]:
"""
Beji and Battjes 1993 experiment
This is an exmaple implemented by PotentialFlow module
"""
from proteus import *
from proteus import (Domain, Context)
from proteus.mprans import SpatialTools as st
from proteus import WaveTools as wt
from proteus import ADR
from math import *
import numpy as np
# ---import .ipynb---
import import_ipynb
import PotentialFlowProblem as PT

opts=Context.Options([
    ("water_level", 0.4, "Height of free surface above seabed"),
    ("tank_dim", (37.7, 0.4), "Dimensions of the tank"),
    ("generation", True, "Generate waves at the left boundary (True/False)"),
    ("free_slip", True, "Should tank walls have free slip conditions "
                        "(otherwise, no slip conditions will be applied)."),
    # waves
    ("wave_gen", True, "Generate waves (True/False)"),
    ("wave_period", 2.0, "Period of the waves"),
    ("wave_height", 0.02, "Height of the waves"),
    ("wave_depth", 0.4, "Wave depth"),
    ("wave_dir", (1.,0.,0.), "Direction of the waves (from left boundary)"),
    ("wave_wavelength", 9.8*2.0**2/(2.0*pi), "Direction of the waves (from left boundary)"), #calculated by FFT
    ("wave_type", 'Linear', "type of wave"),
    ("fast", False, "switch for fast cosh calculations in WaveTools"),
    # mesh refinement
    ("refinement", False, "Gradual refinement"),
    ("he", 0.01, "Set characteristic element size"),
    # numerical options
    ("gen_mesh", True, "True: generate new mesh every time. False: do not generate mesh if file exists"),
    ("use_gmsh", False, "True: use Gmsh. False: use Triangle/Tetgen"),
    ("T", 60.0, "Simulation time"),
    ("dt_init", 0.001, "Initial time step"),
    ("cfl", 0.33 , "Target cfl"),
    ("nsave",  10, "Number of time steps to save per second"),
    ("dt_out", 0.001, "output time interval"),   #---tsao add---#
    ])

# -----------------------------------------------------------------------------
def mean_u():
    '''
    This code is for computing mean velocity of the wave generated by WaveTools, i.e. the velocity of a piston-type wavemaker
    Use Simpson's rule to integrate velocity along the y direction
    '''
    u_paddle = []
    # v_paddle = []
    for i in range(ny):
        u_paddle.append(wave.u([0.,y[i],0.],t)[0])
        # v_paddle.append(wave.u([0.,y[i],0.],t)[1])
    num = integrate.simpson(u_paddle, y)
    return num/(y[-1]-y[0])
# -----------------------------------------------------------------------------
        
# waves
if opts.wave_gen is True:
    period = opts.wave_period
    omega = 2*np.pi/opts.wave_period
    height = opts.wave_height
    mwl = opts.water_level
    depth = opts.wave_depth
    direction = opts.wave_dir
    # waves = wt.MonochromaticWaves(period=period,
    #                               waveHeight=height,
    #                               mwl=mwl,
    #                               depth=depth,
    #                               g=np.array([0., -9.81, 0.]),
    #                               waveDir=direction,
    #                               wavelength=opts.wave_wavelength,
    #                               waveType=opts.wave_type,
    #                               #Ycoeff=np.array(opts.Ycoeff),
    #                               #Bcoeff=np.array(opts.Bcoeff),
    #                               #Nf=len(opts.Bcoeff),
    #                               fast=opts.fast)
    # wavelength = waves.wavelength
    # # print(wavelength)
    # print(waves.eta([0.,0.,0.],0.))
    
    x0 = 5.
    waves = wt.SolitaryWaveFenton(waveHeight=height,
                                  mwl=mwl,
                                  depth=depth,
                                  g=np.array([0., -9.81, 0.]),
                                  waveDir=direction,
                                  #trans = np.zeros(3,"d"),
                                  trans = np.array([x0, 0., 0.]),
                                  fast = opts.fast)
    print(waves.eta([5.,0.,0.],0.))
    print(waves.eta([37.7,0.,0.],0.))

# tank options
waterLevel = opts.water_level
tank_dim = opts.tank_dim

# ----- DOMAIN ----- #
domain = Domain.PlanarStraightLineGraphDomain()

# refinement
he = opts.he
smoothing = he*3.

# ----- TANK ------ #
boundaryOrientations = {'y-': np.array([0., -1.,0.]),
                        'y+': np.array([0., +1.,0.]),
                        'x-': np.array([-1., 0.,0.]),
                        'x+': np.array([1., 0.,0.]),
}

boundaryTags = {'y-': 1,
                'y+': 2,
                'x-': 3,
                'x+': 4,
}

vertices = [( 0.00, 0.00),
            ( 6.00, 0.00),
            (12.00, 0.30),
            (14.00, 0.30), 
            (17.00, 0.00),
            (37.70, 0.00), 
            (37.70, 0.40),
            ( 0.00, 0.40)]

vertexFlags = [boundaryTags['y-'], #( 0.00, 0.00),
               boundaryTags['y-'], #( 6.00, 0.00),
               boundaryTags['y-'], #(12.00, 0.30),
               boundaryTags['y-'], #(14.00, 0.30), 
               boundaryTags['y-'], #(17.00, 0.00), 
               boundaryTags['y-'], #(37.70, 0.00), 
               boundaryTags['y-'], #(37.70, 0.40), 
               boundaryTags['y+'],] #( 0.00, 0.40)

nVertices = len(vertices)
segments = [(i, (i+1)%nVertices) for i in range(nVertices)]
segmentFlags = [boundaryTags['y-'],
                boundaryTags['y-'],
                boundaryTags['y-'],
                boundaryTags['y-'],
                boundaryTags['y-'],
                boundaryTags['x+'],
                boundaryTags['y+'],
                boundaryTags['x-'],]

regions = [[ 0.1, 0.1]]
regionFlags=np.array([1])

tank = st.CustomShape(domain,
                      vertices=vertices,
                      vertexFlags=vertexFlags,
                      segments=segments,
                      segmentFlags=segmentFlags,
                      regions=regions,
                      regionFlags=regionFlags,
                      boundaryTags=boundaryTags,
                      boundaryOrientations=boundaryOrientations)

tank.facets = np.array([[[i for i in range(len(segments))]]])

# ----- ASSEMBLE DOMAIN ----- #
domain.MeshOptions.he = he
domain.MeshOptions.use_gmsh = opts.use_gmsh
domain.MeshOptions.genMesh = opts.gen_mesh

st.assembleDomain(domain)
domain.writePoly("meshBeji")

# physical quentities
g = 9.81

# domain.MeshOptions.triangleOptions = "VApq30Dena%8.8f" % (old_div((he ** 2), 2.0),)
# domain.MeshOptions.triangleFlag=0

# test if I can use a class from PotentialFlowProblem (restart the kernel after modifying the .ipynb)
ok = PT.Hi()
print(ok.hello())

importing Jupyter notebook from PotentialFlowProblem.ipynb
0.019999999984880466
3.533468059492906e-15
hello PotentialFlowProblem as PT
